In [8]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [30]:
# Read the drive time matrix from CSV
drive_time_matrix = pd.read_csv('data/matrix_duration_matrix_DRIVE.csv', index_col=0)
# Convert drive time values from seconds to minutes
drive_time_matrix = drive_time_matrix.astype('float64') / 60

drive_time_matrix.columns = drive_time_matrix.columns.astype(int)
drive_time_matrix.index = drive_time_matrix.index.astype(int)

caregivers = pd.read_csv('data/caregivers.csv', index_col=0)

tasks = pd.read_csv('data/hemtjanst_tasks.csv', index_col=0)
tasks = tasks[(tasks['TaskType'] == 'Hemtjänst')]
tasks['ClientID'] = tasks['ClientID'].astype(int)

clients = pd.read_csv('data/clients.csv', index_col=0)
caregivers['Attributes'] = caregivers['Attributes'].apply(lambda x: np.array(eval(x)))
clients['Requirements'] = clients['Requirements'].apply(lambda x: np.array(eval(x)))
# Alternatively, we could keep track of which entries were NA
# tasks['is_client_id_na'] = tasks['ClientID'].isna()
display(tasks)
display(caregivers)
display(clients)

,ClientID,StartTime,EndTime,TaskType,PlannedCaregiverID
ID,,,,,
4693295,1661,2025-03-05T07:30:00+01:00,2025-03-05T07:50:00+01:00,Hemtjänst,1146
4721787,679,2025-03-05T07:30:00+01:00,2025-03-05T08:00:00+01:00,Hemtjänst,33
4730882,1569,2025-03-05T07:30:00+01:00,2025-03-05T08:15:00+01:00,Hemtjänst,1180
4882374,1612,2025-03-05T07:30:00+01:00,2025-03-05T07:40:00+01:00,Hemtjänst,1163
4725053,1474,2025-03-05T07:30:00+01:00,2025-03-05T08:00:00+01:00,Hemtjänst,1284
...,...,...,...,...,...
4884848,1704,2025-03-05T22:10:00+01:00,2025-03-05T22:30:00+01:00,Hemtjänst,1364
4730192,1349,2025-03-05T22:10:00+01:00,2025-03-05T22:20:00+01:00,Hemtjänst,1264
4730557,978,2025-03-05T22:10:00+01:00,2025-03-05T22:25:00+01:00,Hemtjänst,979


,ModeOfTransport,Attributes,EarliestStartTime,LatestEndTime,StartLocation,EndLocation,RequiresBreak
ID,,,,,,,
1264,pedestrian,"[1, 0]",2025-03-05T16:00:00+01:00,2025-03-05T22:30:00+01:00,Home,Home,True
979,car,"[1, 1]",2025-03-05T16:00:00+01:00,2025-03-05T22:30:00+01:00,HQ,HQ,True
36,car,"[0, 0]",2025-03-05T10:00:00+01:00,2025-03-05T17:00:00+01:00,HQ,HQ,True
33,car,"[0, 0]",2025-03-05T07:30:00+01:00,2025-03-05T16:00:00+01:00,HQ,HQ,True
1335,pedestrian,"[1, 0]",2025-03-05T07:30:00+01:00,2025-03-05T16:15:00+01:00,Home,Home,True
949,car,"[1, 0]",2025-03-05T07:30:00+01:00,2025-03-05T16:00:00+01:00,HQ,HQ,True
1376,bicycle,"[0, 0]",2025-03-05T16:00:00+01:00,2025-03-05T22:30:00+01:00,Home,Home,True
1183,pedestrian,"[0, 0]",2025-03-05T07:30:00+01:00,2025-03-05T16:00:00+01:00,Home,Home,True
1167,pedestrian,"[0, 0]",2025-03-05T16:00:00+01:00,2025-03-05T22:30:00+01:00,Home,Home,True


,Requirements
ID,
156,"[0, 0]"
156,"[0, 0]"
156,"[0, 0]"
174,"[0, 0]"
174,"[0, 0]"
...,...
1704,"[0, 0]"
1709,"[0, 0]"
1712,"[0, 0]"


In [39]:
# Convert time strings to datetime objects and extract minutes since midnight
tasks['start_minutes'] = pd.to_datetime(tasks['StartTime']).dt.hour * 60 + pd.to_datetime(tasks['StartTime']).dt.minute
tasks['end_minutes'] = pd.to_datetime(tasks['EndTime']).dt.hour * 60 + pd.to_datetime(tasks['EndTime']).dt.minute
tasks['duration_minutes'] = tasks['end_minutes'] - tasks['start_minutes']
caregivers['start_minutes'] = pd.to_datetime(caregivers['EarliestStartTime']).dt.hour * 60 + pd.to_datetime(caregivers['EarliestStartTime']).dt.minute
caregivers['end_minutes'] = pd.to_datetime(caregivers['LatestEndTime']).dt.hour * 60 + pd.to_datetime(caregivers['LatestEndTime']).dt.minute

In [53]:
def build_home_care_model_dfbased(
    caregivers: pd.DataFrame,           # ID,ModeOfTransport,Attributes,start_minutes,end_minutes,StartLocation,EndLocation,RequiresBreak
    tasks: pd.DataFrame,                # ID,ClientID,start_minutes,end_minutes,duration_minutes,TaskType,PlannedCaregiverID
    clients: pd.DataFrame,              # ID,Requirements
    drive_time_matrix: pd.DataFrame,    # Drive time matrix from client col to client row. Uses client ID as indexes. Index 0 is the HQ.
    M = 1_000_000                       # Big M
):
    K = caregivers.index.tolist()
    V = tasks.index.tolist()
    model = gp.Model('HomeCare')
    
    # For each caregiver, gather only the patients that caregiver k can serve,
    # then define the augmented node set (start, qualified patients, end).
    caregiver_tasks = {}
    for k in K:
        caregiver_attributes = caregivers.loc[k, 'Attributes']
        qualified_patients = clients[clients['Requirements'].apply(lambda req: np.dot(req, caregiver_attributes) == 0)].index.tolist()
        # Filter tasks to only include the ones with these qualified patients
        caregiver_tasks[k] = tasks[tasks['ClientID'].isin(qualified_patients)].index.tolist()
        
    # ---- 2. Decision Variables

    # x[k, i, j] = 1 if caregiver k goes directly from i to j, else 0.
    # Skip arcs into sigma[k] or out of tau[k], and skip i->i.
    x = {}
    for k in K:
        for i in V:
            # Add route to the start and end nodes
            x[k, "start", i] = model.addVar(vtype=GRB.BINARY, name=f'x^{k}_start_{i}')
            x[k, i, "end"] = model.addVar(vtype=GRB.BINARY, name=f'x^{k}_{i}_end')
            for j in V:
                if i != j:
                    x[k, i, j] = model.addVar(vtype=GRB.BINARY, name=f'x^{k}_{i}_{j}')
    
    # t[k,i] = arrival time of caregiver k at node i
    t = {}
    for k in K:
        t[k, "start"] = model.addVar(vtype=GRB.CONTINUOUS, name=f't^{k}_start')
        t[k, "end"] = model.addVar(vtype=GRB.CONTINUOUS, name=f't^{k}_end')
        for i in V:
            t[k, i] = model.addVar(vtype=GRB.CONTINUOUS, name=f't^{k}_{i}')
            
    model.update()
    
    # ---- 3. Objective Function
    # Minimize time bewteen start and end nodes for all caregivers
    model.setObjective(gp.quicksum(t[k, "end"] - t[k, "start"] for k in K), GRB.MINIMIZE)
    
    # ---- 4. Constraints
    # (V2) Each task is visited exactly once by exactly one caregiver
    for i in V:
        model.addConstr(gp.quicksum(x[k, i, j] for k in K for j in V if i != j) == 1, name=f'UniqueVisit[{i}]')
    
    # (V3) Flow conservation for each caregiver k
    for k in K:
        for i in V:
            model.addConstr(
                gp.quicksum(x[k, i, j] for j in V + ["end"] if i != j) - 
                gp.quicksum(x[k, j, i] for j in V + ["start"] if i != j) == 0, 
                name=f'Flow[{k},{i}]')
    
    # (V4) Route completion (start and end usage) for each caregiver
    # Only need to fix this for the start node, since flow conservation
    # and one visit per task ensures that the end node is also correctly handled
    for k in K:
        model.addConstr(gp.quicksum(x[k, "start", i] for i in V) <= 1, name=f'StartBalance[{k}]')
    
    # (V6) Only visit patients that the caregiver is qualified to visit
    model.addConstr(
        gp.quicksum(x[k, i, j] for k in K for j in V for i in V + ["start"] 
                    if j not in caregiver_tasks[k] and i != j) == 0,
        name='Qualification'
    )
    
    # (V7-V8) Arriving on time
    for k in K:
        for i in V:
            start_minutes = tasks.loc[i, 'start_minutes']
            end_minutes = tasks.loc[i, 'end_minutes']
            duration_minutes = tasks.loc[i, 'duration_minutes']
            model.addConstr(t[k, i] >= start_minutes, name=f'Earliest[{k},{i}]')
            model.addConstr(t[k, i] <= end_minutes - duration_minutes, name=f'Latest[{k},{i}]')
    
    # (V9) Temporal feasibility
    for k in K:
        model.addConstr(t[k, "end"] >= t[k, "start"], name=f'TemporalFeasibility[{k}]')
        for i in V + ["start"]:
            for j in V + ["end"]:
                if i != j and not (i == "start" and j == "end"):
                    # Calculate travel time based on locations
                    if i == "start":
                        # From start location to task
                        travel_time = 0 if caregivers.loc[k, 'StartLocation'] == "Home" else \
                                    drive_time_matrix.loc[0, tasks.loc[j, 'ClientID']]
                        service_time = 0
                    elif j == "end":
                        # From task to end location
                        travel_time = 0 if caregivers.loc[k, 'EndLocation'] == "Home" else \
                                    drive_time_matrix.loc[tasks.loc[i, 'ClientID'], 0]
                        service_time = tasks.loc[i, 'duration_minutes']
                    else:
                        # From task to task
                        travel_time = drive_time_matrix.loc[tasks.loc[i, 'ClientID'], tasks.loc[j, 'ClientID']]
                        service_time = tasks.loc[i, 'duration_minutes']
                    
                    # Add the constraint
                    model.addConstr(
                        t[k, j] >= t[k, i] + travel_time + service_time - M * (1 - x[k, i, j]),
                        name=f'TimeLink[{k},{i}->{j}]'
                    )
    return model, x, t

In [44]:
# Subset with only car caregivers
car_caregivers = caregivers[caregivers['ModeOfTransport'] == 'car']
car_caregiver_ids = car_caregivers.index.tolist()
car_tasks = tasks[tasks['PlannedCaregiverID'].isin(car_caregiver_ids)]

print(f"Number of car caregivers: {len(car_caregivers)}")
print(f"Number of tasks assigned to car caregivers: {len(car_tasks)}")

Number of car caregivers: 17
Number of tasks assigned to car caregivers: 155


In [50]:
model_car, x_car, t_car = build_home_care_model_dfbased(car_caregivers, car_tasks, clients, drive_time_matrix)


KeyboardInterrupt



In [ ]:
model_car.optimize()

In [51]:
# Subset with only caregivers 33, 949, 1108 and their tasks
subset = [33, 949, 1108]
caregivers_subset = caregivers.loc[subset]
tasks_subset = tasks[tasks['PlannedCaregiverID'].isin(subset)]

print(f"Number of caregivers in subset: {len(caregivers_subset)}")
print(f"Number of tasks assigned to subset caregivers: {len(tasks_subset)}")

Number of caregivers in subset: 3
Number of tasks assigned to subset caregivers: 27


In [54]:
model_subset, x_subset, t_subset = build_home_care_model_dfbased(caregivers_subset, tasks_subset, clients, drive_time_matrix)

In [55]:
model_subset.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2545 rows, 2355 columns and 13641 nonzeros
Model fingerprint: 0x97027556
Variable types: 87 continuous, 2268 integer (2268 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 162 rows and 1326 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -


In [29]:
def extract_model_metrics(model, x, t, K, V, s, sigma, tau):
    """
    Extract key metrics from an optimized model.
    
    Parameters:
    -----------
    model : Gurobi model
        The optimized model
    K : list
        List of caregivers
    V : list
        List of patients/clients
    s : dict
        Service times for each node
    t : dict
        Time variables from the model
    x : dict
        Flow variables from the model
    sigma : dict
        Start depot for each caregiver
    tau : dict
        End depot for each caregiver
        
    Returns:
    --------
    tuple
        - Dictionary containing metrics for each caregiver:
          - total_time: end time - start time
          - productive_time: sum of service times for visited patients
          - utilization_rate: productive_time / total_time
        - Dictionary containing aggregated metrics:
          - total_time: sum of all caregivers' total time
          - productive_time: sum of all caregivers' productive time
          - overall_utilization_rate: total productive time / total time
    """
    
    if model.Status != 2:  # Check if model is optimized (status 2 is optimal)
        print("Warning: Model is not optimized. Results may be incorrect.")
    
    caregiver_metrics = {}
    
    total_all_time = 0
    total_productive_time = 0
    
    for k in K:
        # Extract values from the model
        t_values = {node: t[(k, node)].X for node in [sigma[k], tau[k]] + [i for i in V if (k, i) in t]}
        x_values = {(i, j): x[(k, i, j)].X for i, j in [(i, j) for i in V + [sigma[k]] 
                                                      for j in V + [tau[k]] 
                                                      if i != j and (k, i, j) in x]}
        
        # Calculate total time (end time - start time)
        total_time = t_values[tau[k]] - t_values[sigma[k]]
        
        # Calculate productive time
        productive_time = 0
        for i in V:
            # Check if patient i is visited by caregiver k
            is_visited = sum(x_values.get((j, i), 0) for j in V + [sigma[k]] if j != i) > 0.5
            if is_visited:
                productive_time += s[i]
        
        caregiver_metrics[k] = {
            'total_time': total_time,
            'productive_time': productive_time,
            'utilization_rate': productive_time / total_time if total_time > 0 else 0
        }
        
        total_all_time += total_time
        total_productive_time += productive_time
    
    # Calculate aggregated metrics
    total_metrics = {
        'total_time': total_all_time,
        'productive_time': total_productive_time,
        'overall_utilization_rate': total_productive_time / total_all_time if total_all_time > 0 else 0
    }
    
    return caregiver_metrics, total_metrics
